# 0. Iportable Globals and Metadata

Global data and helper functions to be imported into each notebook.

The functions here are outlined more clearly in 1.Globals_and_Metadata.ipynb

In [1]:
import requests
import requests_cache
from pprint import pprint
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
requests_cache.install_cache('demo_cache')

In [2]:
ds = '499682b1-3174-493f-ba1a-368b4636708e'  # ADMIN 2 level data

<a id='s0'></a>
# Obtain metadata for admin-0 to admin-2 level

Retrieve json mappings of id values (e.g. iso codes or integers) for admin0 to admin2 level, and map them to english names for selectors and resources to build dynamic sentences.

In [3]:
def get_admin0_json(urlCarto="https://wri-01.carto.com/api/v2/sql"):
    """return an alphabetical json of mappings of iso and country names
    e.g.
    [{'iso': 'AFG', 'name': 'Afghanistan'},
     {'iso': 'XAD', 'name': 'Akrotiri and Dhekelia'},
     {'iso': 'ALA', 'name': 'Åland'},
     ...
    ]
    
    """
    sql = ("SELECT iso, country as name FROM umd_nat_staging "
            "GROUP BY iso, name "
            "ORDER BY name "
          )
    r = requests.get(urlCarto, params={"q": sql})
    return r.json().get('rows', None)
 

def get_admin1_json(iso, urlCarto="https://wri-01.carto.com/api/v2/sql"):
    """return an alphabetical json of mappings of admin1 for a country
    e.g. for iso='GBR'
        [{'adm1': 1, 'name': 'England'},
         {'adm1': 2, 'name': 'Northern Ireland'},
         {'adm1': 3, 'name': 'Scotland'},
         {'adm1': 4, 'name': 'Wales'}]
    """
    sql = ("SELECT id1 as adm1, region as name "
            "FROM umd_subnat_staging "
            f"WHERE iso = '{iso}' "
            "and year = 2001 "
            "and thresh = 30 "
            "ORDER BY name "
          )
    r = requests.get(urlCarto, params={"q": sql})
    return r.json().get('rows', None)
   

def get_admin2_json(iso, adm1, urlCarto="https://wri-01.carto.com/api/v2/sql"):
    """ 
    e.g. response for iso='GBR' adm1=1
    [{'adm2': 1, 'name': 'Barking and Dagenham'},
     {'adm2': 2, 'name': 'Bath and North East Somerset'},
     ...
     ]

    """
    sql = ("SELECT id_2 as adm2, name_2 as name "
            "FROM gadm28_adm2 "
            f"WHERE iso = '{iso}' "
            f"AND id_1 = {adm1} "
            "ORDER BY name "
          )
    r = requests.get(urlCarto, params={"q": sql})    
    return r.json().get('rows', None)

In [4]:
tmp = get_admin0_json()
iso_to_countries = {}
for row in tmp:
    iso_to_countries[row.get('iso')] = row.get('name')

# Get info on an areas shape

Use the Geostore service to find the geometry, bounding box or additional info on a given `iso` or `iso/admin1` area if required (e.g. for centering the map).

Note - if similar info is needed for admin2 areas I will have to go back and create a query based on a Carto table, as admin-2 level data does not exist in the geostore service yet.

In [5]:
iso = "GBR"
adm1 = None 

if not adm1:
    url = f"https://api.resourcewatch.org/v1/geostore/admin/{iso}"
else:
    url = f"https://api.resourcewatch.org/v1/geostore/admin/{iso}/{adm1}"
    
r = requests.get(url)

roi = r.json()

In [6]:
def create_bounding_poly(bbox):
    """Create a bounding polygon from a returned bbox of the geostore service
        (- the old api produced shapes in this format).
    """
    lat1, lon1, lat2, lon2  = bbox
    return [[lat1,lon1], [lat1,lon2],[lat2,lon2],[lat2,lon1] ,[lat1,lon1]]

# Set up global (scope) data sources for the Country Widgets 

There is some work that needs to be done in populating the Location menu, not all options should be always available: some of them will need to be iso dependent. E.g. `Biomes` should only appear in the menu if the ISO is set to `BRA`


In [7]:
polynames = {
                'All Region':'gadm28',
                'Biomes':'bra_biomes',
                'Mining':'mining',
                'Protected Areas':'wdpa',
                'Primary Forests':'primary_forest',
                'Mining in Primary Forests':'primary_forest__mining',
                'Protected Areas in Primary Forests':'primary_forest__wdpa',
                'Intact Forest Landscapes':'ifl_2013',
                'Protected Areas in Intact Forest Landscapes':'ifl_2013__wdpa',
                'Mining in Intact Forest Landscapes':'ifl_2013__mining',
                'Indigenous Lands':'landmark',
                'Indigenous Lands in Primary Forests':'primary_forest__landmark',
                'Plantations':'plantations',
                'Mining in Plantation Areas':'plantations__mining',
                'Protected areas in Plantations':'plantations__wdpa',
                'Indigenous Lands in Plantations':'plantations__landmark',
            }

plantation_type_dict = {
    
                         'Large industrial plantation': 'large industrial',
                         'Clearing/ very young plantation': 'clearing or very young',
                         'Mosaic of medium-sized plantations': 'medium-sized',
                         'Mosaic of small-sized plantations': 'small sized'

                        }

plantation_species_dict = {
                            'Wood fiber / timber': 'wood fiber and timber',
                            'Wood fiber / timber mix': 'mixed wood fiber and timber',
                            'Recently cleared': 'recently cleared',
                            'Unknown': 'unknown',
                            'Fruit': 'fruit tree',
                            'Fruit mix': 'mixed fruit tree',
                            'Oil palm': 'palm tree',
                            'Other mix': 'mixed',
                            'Other': 'other',
                            'Rubber': 'rubber tree'
                            }

extent_year_dict = {
    
         2000:'area_extent_2000',
         2010:'area_extent'

        }

tabs = {
         "summary":"Summary",
         "land_cover":"Land Cover",
         "land_use": "Land Use",
         "forest_change": "Forest Change",
         "conservation":'Conservation',
         "people":"People",
         "climate":"Climate"
        }

# Dynamic Whitelists Tests

In [8]:
iso = "SAU"
adm1 = None
adm2 = None

In [9]:
def get_whitelist(iso, adm1=None, adm2=None):
    if adm2:
        sql = (f"SELECT polyname FROM {ds} "
               "WHERE thresh = 0 AND polyname is not 'gadm28' "
               f"AND iso = '{iso}' AND adm1={adm1} AND adm2={adm2} "
               "GROUP BY polyname")
    elif adm1:
        sql = (f"SELECT polyname FROM {ds} "
               "WHERE thresh = 0 AND polyname is not 'gadm28' "
               f"AND iso = '{iso}' AND adm1={adm1} "
               "GROUP BY polyname")
    else:
        sql = (f"SELECT polyname FROM {ds} "
               "WHERE thresh = 0 AND polyname is not 'gadm28' "
               f"AND iso = '{iso}' "
               "GROUP BY polyname")
    return sql
    

In [10]:
# Create whitelist for each polyname

url = f"https://production-api.globalforestwatch.org/v1/query/{ds}"
sql = get_whitelist(iso=iso, adm1=adm1, adm2=adm2)

properties = {"sql": sql}
r = requests.get(url, params = properties)
data = r.json().get('data')


In [11]:
unique_polys = []

for d in data:
    if d.get('polyname') not in unique_polys:
        unique_polys.append(d.get('polyname'))


### Create general Whitelist for each polyname

In [12]:
# Create whitelist for each polyname

url = f"https://production-api.globalforestwatch.org/v1/query/{ds}"
sql = (f"SELECT iso, polyname FROM {ds} WHERE thresh = 0 AND polyname is not 'gadm28'")

properties = {"sql": sql}
r = requests.get(url, params = properties)

data = r.json().get('data')


In [13]:
whitelist_by_polys = []

for d in data:
    if d.get('polyname') not in whitelist_by_polys:
        whitelist_by_polys.append(d.get('polyname'))

        
polyname_config = {}
for u in whitelist_by_polys:
    tmp_isos = []
    for d in data:
        if d.get('polyname') == u and d.get('iso') not in tmp_isos:
            tmp_isos.append(d.get('iso'))
    polyname_config[u] = tmp_isos


###  Alternate Whitelist

This query can be used to in place of the previous one to assess whether polynames contain non-zero data in addition to building a whitelist.


In [14]:
iso = "BRA"
adm1 = 4
adm2 = None

In [15]:
def get_alt_whitelist(iso, adm1=None, adm2=None):
    if adm2:
        sql = ("SELECT polyname, SUM(area_extent_2000) as total_extent_2000, "
               "SUM(area_extent) as total_extent_2010, "
               "SUM(area_gain) as total_gain, "
               f"SUM(year_data.area_loss) as total_loss FROM {ds} "
               "WHERE thresh = 0 "
               f"AND iso = '{iso}' AND adm1={adm1} AND adm2={adm2} "
               "GROUP BY polyname")
    elif adm1:
        sql = ("SELECT polyname, SUM(area_extent_2000) as total_extent_2000, "
               "SUM(area_extent) as total_extent_2010, "
               "SUM(area_gain) as total_gain, "
               f"SUM(year_data.area_loss) as total_loss FROM {ds} "
               "WHERE thresh = 0 "
               f"AND iso = '{iso}' AND adm1={adm1} "
               "GROUP BY polyname")
    else:
        sql = ("SELECT polyname, SUM(area_extent_2000) as total_extent_2000, "
               "SUM(area_extent) as total_extent_2010, "
               "SUM(area_gain) as total_gain, "
               f"SUM(year_data.area_loss) as total_loss FROM {ds} "
               "WHERE thresh = 0 "
               f"AND iso = '{iso}' "
               "GROUP BY polyname")
    return sql
    

In [16]:
# Create whitelist for each polyname

url = f"https://production-api.globalforestwatch.org/v1/query/{ds}"
sql = get_alt_whitelist(iso=iso, adm1=adm1, adm2=adm2)

properties = {"sql": sql}
r = requests.get(url, params = properties)

data = r.json().get('data')
